In [1]:
# Setup
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
import time
import warnings
import torch.nn as nn
import pickle
from dataset_loader import *
from utils import *
import pandas as pd
from datasets import Dataset, DatasetDict
from retrain_utils import *

SEED = 84
torch.manual_seed(SEED)
warnings.simplefilter("ignore")

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
dataset_name = 'sentiment'
ds = get_ds(dataset_name)

In [4]:
#model_name ='google/electra-small-discriminator'
model_name =  'huawei-noah/TinyBERT_General_4L_312D'
folder_name = 'tinybert'

In [5]:
replaced_df = retrainUtils(model_name, dataset_name).replace_sentences()

In [6]:
new_train = pd.concat([ds['train'].to_pandas(), replaced_df], ignore_index=True)
ds['train'] = Dataset.from_pandas(new_train)

In [7]:
set_seed()
model = load_model(model_name)
tokenized_data = tokenize_dataset(ds, tokenizer_name=model_name, max_length = 64)
train(model, tokenized_data, path=f'{folder_name}/{dataset_name}/check', num_train_epochs=10)
model.save_pretrained(f'{folder_name}/{dataset_name}/updated_model')

Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertForSequenceClassification: ['fit_denses.3.weight', 'cls.predictions.decoder.weight', 'fit_denses.2.bias', 'fit_denses.0.bias', 'fit_denses.1.weight', 'fit_denses.1.bias', 'fit_denses.0.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'fit_denses.3.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'fit_denses.4.weight', 'fit_denses.2.weight', 'fit_denses.4.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a

  0%|          | 0/9777 [00:00<?, ?ex/s]

  0%|          | 0/2087 [00:00<?, ?ex/s]

***** Running training *****
  Num examples = 9777
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3060


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.397804,0.823191
2,0.455600,0.396811,0.822233
3,0.455600,0.404085,0.841878
4,0.294200,0.458214,0.843316
5,0.198500,0.490111,0.838524
6,0.198500,0.493688,0.839483
7,0.134000,0.664571,0.836128
8,0.134000,0.703238,0.837087
9,0.100200,0.748798,0.840920
10,0.072200,0.774139,0.834691


***** Running Evaluation *****
  Num examples = 2087
  Batch size = 32
Saving model checkpoint to tinybert/sentiment/check/checkpoint-306
Configuration saved in tinybert/sentiment/check/checkpoint-306/config.json
Model weights saved in tinybert/sentiment/check/checkpoint-306/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2087
  Batch size = 32
Saving model checkpoint to tinybert/sentiment/check/checkpoint-612
Configuration saved in tinybert/sentiment/check/checkpoint-612/config.json
Model weights saved in tinybert/sentiment/check/checkpoint-612/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2087
  Batch size = 32
Saving model checkpoint to tinybert/sentiment/check/checkpoint-918
Configuration saved in tinybert/sentiment/check/checkpoint-918/config.json
Model weights saved in tinybert/sentiment/check/checkpoint-918/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2087
  Batch size = 32
Saving model checkpoint to tinybert/sentiment/c

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [9]:
def predict_sentences(sentences):
    encoded = [[101] +[tokenizer._convert_token_to_id_with_added_voc(token) for token in tokens] + [102]         
               for tokens in sentences]
    #encoded = tokenizer.encode(sentences, add_special_tokens=True, return_tensors="pt").to(device)
    to_pred = torch.tensor(encoded, device=device)
    outputs = traced_model(to_pred)[0]
    print(outputs)
    return torch.argmax(outputs, dim=1).cpu().numpy()

### save best model's folder as 'model'

In [23]:
model = AutoModelForSequenceClassification.from_pretrained(f'{folder_name}/{dataset_name}/model', num_labels=2, torchscript=True).to(device)
model = model.eval()

loading configuration file tinybert/sentiment/model/config.json
Model config BertConfig {
  "_name_or_path": "tinybert/sentiment/model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pre_trained": "",
  "problem_type": "single_label_classification",
  "structure": [],
  "torch_dtype": "float32",
  "torchscript": true,
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file tinybert/sentiment/model/pytorch_model.bin
All model checkpoint weights were used when initializ

In [6]:
tokenized_data = tokenize_dataset(ds, tokenizer_name=model_name, max_length = 64)

  0%|          | 0/8346 [00:00<?, ?ex/s]

  0%|          | 0/2087 [00:00<?, ?ex/s]

In [32]:
train(model, tokenized_data, path=f'{folder_name}/{dataset_name}', evaluate = True)

PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 8346
  Batch size = 32


{'eval_loss': 0.2261132448911667,
 'eval_accuracy': 0.917685118619698,
 'eval_runtime': 3.2137,
 'eval_samples_per_second': 2597.032,
 'eval_steps_per_second': 81.216}

In [23]:
encoded = tokenizer.encode("i love movies", add_special_tokens=True, return_tensors="pt").to(device)
traced_model = torch.jit.trace(model, encoded)

In [24]:
torch.jit.save(traced_model, f"{folder_name}/{dataset_name}/traced.pt")

In [17]:
traced_model = torch.jit.load(f"{folder_name}/{dataset_name}/traced.pt")

In [25]:
predict_sentences([tokenizer.tokenize("i hate movies")])

tensor([[ 2.1096, -2.0074]], device='cuda:0', grad_fn=<AddmmBackward0>)


array([0])

In [29]:
predict_sentences([tokenizer.tokenize("i love movies")])

tensor([[-2.4865,  2.3588]], device='cuda:0', grad_fn=<AddmmBackward0>)


array([1])